In [ ]:
import os
import sys
from time import sleep

import numpy as np

import GCode
import GRBL


# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

In [9]:
# Standard brick dimensions.
BrickHeight = 65  # mm
BrickLength = 225  # mm
BrickDepth = 12.5  # mm
BrickRatio = 215 / 65  # [dimensionless]


In [10]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [7]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 5

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows
L_Block_Brick = H_Block_Brick * BrickRatio, 4


# Code:

# Default Line

Sane default for the ```draw_line``` command. Designed so that ```draw_line()``` does something that is easly measureable.

Draw a 30-60-90 triangle.

From: https://www.dummies.com/education/math/calculus/how-to-work-with-30-60-90-degree-triangles.

If you look at the 30:60:90-degree triangle in radians, it translates to the following:

$$\frac{\pi}{6}:\frac{\pi}{3}:\frac{\pi}{2}$$

In any 30-60-90 triangle, you see the following:

- The shortest leg is across from the 30-degree angle.

- The length of the hypotenuse is always two times the length of the shortest leg.

- You can find the long leg by multiplying the short leg by the square root of 3.

If you know one side of a 30-60-90 triangle, you can find the other two by using shortcuts. Here are the three situations you come across when doing these calculations:

- **Type 1**: You know the short leg (the side across from the 30-degree angle). Double its length to find the hypotenuse. You can multiply the short side by the square root of 3 to find the long leg.

- **Type 2**: You know the hypotenuse. Divide the hypotenuse by 2 to find the short side. Multiply this answer by the square root of 3 to find the long leg.

- **Type 3**: You know the long leg (the side across from the 60-degree angle). Divide this side by the square root of 3 to find the short side. Double that figure to find the hypotenuse.

Let:

- a: Shortest Side. Opposite 30$^o$ ($\frac{\pi}{6}$)

In [12]:
a = 3


In [14]:
default_points = np.array(
    [
        [0, 0],  # Start at origin.
        [a * np.sqrt(3), 0],  # Draw long side along X axis.
        [a * np.sqrt(3), a],  # Draw the short side parallel to Y axis.
        [0, 0],  # Return to origin. Draw hypotenuse.
    ]
)
default_points


array([[0.        , 0.        ],
       [5.19615242, 0.        ],
       [5.19615242, 3.        ],
       [0.        , 0.        ]])

Default settings for feed rate and laser power:

In [ ]:
default_feed = 300  # mm/n
default_power = 150  # [dimensionless]


In [36]:
class Line(GCode.GCode):
    def __init__(
        self,
        points=default_points,
        feed=default_feed,
        power=default_power,
        dynamic_power=True,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.points = points
        self.feed = feed
        self.power = power
        self.dynamic_power = dynamic_power

    def generate_gcode(self):
        self.buffer = list()
        self.G0(F=60)  # 60 mm / min = 1 mm / sec
        self.G1(F=60)  # 60 mm / min = 1 mm / sec
        self.M3(
            S=1
        )  # Set laser power so that movement can be seen, but does nothing.
        self.G28()  # "Home"
        self.G21()  # Metric Units
        self.G90()  # Absolute positioning.
        self.G92(
            X=0, Y=0, Z=0
        )  # The cliche, I forgot why I added it. It works. Don't touch it.

        self.G0(X=self.points[0, 0], Y=self.points[0, 1])


In [37]:
default_points


array([[0.        , 0.        ],
       [5.19615242, 0.        ],
       [5.19615242, 3.        ],
       [0.        , 0.        ]])

In [38]:
self = Line()
self.G0(X=0, Y=0)
self.G0(X=10, Y=5)


In [39]:
type(self.buffer)


list

In [41]:
self.buffer


['G0 X0 Y0', 'G0 X10 Y5']

In [42]:
self.generate_gcode()
